In [1]:
import numpy as np
import strawberryfields as sf
from strawberryfields.ops import *
import tensorflow as tf
import random as rd
import pickle

import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input
from datetime import datetime
from tensorflow.keras import optimizers
from scipy.special import comb
from tensorflow.keras.initializers import RandomUniform, Identity
from tensorflow.keras.layers import Dense, ReLU
from tensorflow.keras import activations
from tensorflow.keras.callbacks import LearningRateScheduler, TensorBoard
import pickle
from tensorflow.keras.optimizers.schedules import PolynomialDecay

In [3]:
# what is the interpretation of this function - could one name it accordingly? Like multiply_3inputs?
# prediction fnction xt, xt-1, xt-2
def function_lag_iris(x1,x2,x3):
    return x1*x2*x3

# random input data
inp=np.random.random_sample(100)*0.5

print("Memristor time lag")
res_mem={}
prove_num=1 #?

# Linear, square und cube functions to predict // Here only cube
for power in [3]:
    dip=power # memory // why call it dip? 
    res_mem={}

    for p in range(prove_num):
        eng = sf.Engine(backend="tf", backend_options={"cutoff_dim": 4})
        
        # create 3 mode circuit
        circuit=sf.Program(3)

        # use random training points
        x_train=np.sqrt(inp)
        
        # translate values to angles -> encoding
        angs=2*np.arccos(x_train)
    
    
        
        print("prova %d su %d" %(p, prove_num)) # Trial 1 of 1 

        # first memoristor phase (random)
        phi1 = tf.Variable(rd.uniform(0.01,1)*2*np.pi, constraint=lambda z: tf.clip_by_value(z, 0, 2*np.pi))

        # x_2 free parameter in phi2, phi2 is memoristor 
        x_2=tf.Variable(rd.uniform(0.01,1),  constraint=lambda z: tf.clip_by_value(z, 0.01, 1)) 
        p1=tf.constant(rd.uniform(0.01,1))
        p2=tf.Variable(rd.uniform(0.01,1))

        # input training data
        phienc = tf.constant(angs)

         # second memoristor phase (random)
        phi3 = tf.Variable(rd.uniform(0.01,1)*2*np.pi, constraint=lambda z: tf.clip_by_value(z, 0, 2*np.pi))
        print("Random choices: ", phi1, phi3, x_2)
        
        phi_1, phi_2,phi_3, phi_enc = circuit.params("phi1","phi2", "phi3", "phienc")
        
        
            
            
        with circuit.context as q:
            Vac     | q[0]
            Fock(1) | q[1]
            Vac     | q[2]
        
            ###----------------------------------------------------------------------
        
            # For memoristot implementation see https://www.nature.com/articles/s41566-022-00973-5 Figure 2e 


            ## encoding MZI
        
            BSgate(np.pi/4, np.pi/2) | (q[0], q[1])
        
            Rgate(phi_enc)           | q[1] # encoding (input in paper)
        
            BSgate(np.pi/4, np.pi/2) | (q[0], q[1])
            
            ## first MZI
        
            BSgate(np.pi/4, np.pi/2) | (q[0], q[1])
        
            Rgate(phi_1)              | q[1] # state preparation
        
            BSgate(np.pi/4, np.pi/2) | (q[0], q[1])
            
             ## second MZI
        
            BSgate(np.pi/4, np.pi/2) | (q[1], q[2])
        
            Rgate(phi_2)              | q[1] # memoristor value
        
            BSgate(np.pi/4, np.pi/2) | (q[1], q[2])
            
            
            ## third MZI
        
            BSgate(np.pi/4, np.pi/2) | (q[0], q[1])
        
            Rgate(phi_3)              | q[1] # state tomography
        
            BSgate(np.pi/4, np.pi/2) | (q[0], q[1])
            
            
        opt = tf.keras.optimizers.Adam(learning_rate=0.003)
        steps = 1
        
        for step in range(steps):
        
            # reset the engine if it has already been executed
            if eng.run_progs:
                eng.reset()
        
            with tf.GradientTape() as tape:
                loss=0
                index=0
                p1=np.zeros(dip) # dip is memory of memristor 
                p2=np.zeros(dip)
                for phi in range(len(phienc)): # 0 - all datapoints
                    t=phi-index*dip # index 0 - 3 // t 0-2
                    if t==dip-1: 
                        index+=1
                    if phi==0: # if memory empty, start with sq 0.5 
                        results = eng.run(circuit, args={"phi1": phi1, "phi3": phi3, "phi2": tf.Variable(tf.acos(np.sqrt(0.5))), "phienc": phienc[phi]})
                    if phi>0: # if not empty use last values until max dip --> memoristor juhu
                        results = eng.run(circuit, args={"phi1": phi1, "phi3": phi3, "phi2": tf.acos(tf.sqrt(np.sum(p1)/dip+x_2*np.sum(p2)/dip)), "phienc": phienc[phi]})

                    # get probabilities, function pred output of quantum circuit
                    prob=results.state.all_fock_probs()

                    # store prob in memory p1 (index at t), prob[0,1,0] is the probability of having 1 photon in mode 1 and 0 in mode 0 and mode 2
                    p1[t]=tf.Variable(np.real(prob[0,1,0])) # store prob in memory p1 (index at t)
                    p2[t]=tf.Variable(np.real(prob[0,0,1]))
        
                    if phi>=2: # should be phi > dip
                        # why is the to the power 6 part left out here? because there is no memristor?
                        f2=function_lag_iris(x_train[phi]**2,x_train[phi-1]**2,x_train[phi-2]**2) # function to predict, current, past, past
                        loss += (abs(f2-prob[0,0,1]))**2 

                        # old functions, discard
                        #f1=x_train[phi]**6
                        #loss += (abs(f2-prob[0,1,0])+abs(f1-prob[0,0,1]))**2

            gradients = tape.gradient(loss, [phi1, phi3, x_2]) # get gradients
            opt.apply_gradients(zip(gradients, [phi1, phi3, x_2]))
            res_mem[('loss', 'tr', step, p, power)]=[loss, phi1.value(), phi3.value(), x_2.value(), inp] # store results of memoristor for log
            print("Loss at step {}: {}".format(step+1, np.real(loss)))
        print("Final loss: %f" %np.real(loss))
        print("Optimal parameters: %f and %f\n x2= %f" %(phi1,phi3, x_2))
    file=open("results_mem_"+str(power)+"_t_lag_iris_lr_lag_try", "wb")
    pickle.dump(res_mem, file)
    file.close()

Memristor time lag
prova 0 su 1
Random choices:  <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=2.5086064> <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=5.664234> <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=0.2614594>
Loss at step 1: 47.10768127441406
Final loss: 47.107681
Optimal parameters: 2.511606 and 5.665037
 x2= 0.258459
